In [13]:
import torch
import random
import sys
sys.path.append('/content/drive/MyDrive/Simple_RLHF')

from google.colab import drive

drive.mount('/content/drive')

from util import TokenizerUtil
tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

(tensor([   0, 9178,   32,    2]), tensor([1, 1, 1, 1]), '<s>how are</s>')

In [14]:
!pip install datasets
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='/content/drive/MyDrive/Simple_RLHF/dataset/train.json', split='train')

#2,4,4切分,取第0部分
dataset = dataset.select(range(15000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=2,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

(7500,
 {'input_ids': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]),
  'labels': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]])})

In [15]:
from google.colab import drive

drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Simple_RLHF')

from transformers import AutoModelForCausalLM
import costom_lora

model_actor = AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b')

costom_lora.insert(model_actor)
costom_lora.count_params(model_actor)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'count_require': 2.21044736, 'count_all': 14.29004288, 'ratio': 0.15468444556549854}


In [16]:
import torch

from transformers import get_scheduler
from accelerate import Accelerator


def f():
    params = []
    params_lora = []
    for name, param in model_actor.named_parameters():
        if not param.requires_grad:
            continue

        if 'lora_A' in name or 'lora_B' in name:
            params_lora.append(param)
            continue

        params.append(param)

    return [{
        'params': params,
        'weight_decay': 0.0,
    }, {
        'params': params_lora,
        'weight_decay': 0.0,
        'lr': 5e-4
    }]


optimizer = torch.optim.Adam(f(), lr=1e-3, betas=(0.9, 0.95))

scheduler = get_scheduler(name='cosine',
                          optimizer=optimizer,
                          num_warmup_steps=0,
                          num_training_steps=100)

accelerator = Accelerator(gradient_accumulation_steps=64,
                          mixed_precision='fp16')

model_actor, loader, optimizer, scheduler = accelerator.prepare(
    model_actor, loader, optimizer, scheduler)

model_actor.train()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (v_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (q_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
            (out_proj): Lora(
              (linear): Linear(in_features=2048, out_features=2048, bias=True)
            )
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)


In [17]:
for i, data in enumerate(loader):
    with accelerator.accumulate(model_actor):
        out = model_actor(**data)
        accelerator.backward(out.loss)

        if accelerator.sync_gradients:
            accelerator.clip_grad_norm_(
                [i for i in model_actor.parameters() if i.requires_grad], 1.0)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    if (i + 1) % 100 == 0:
        lr = optimizer.param_groups[0]['lr']
        print(i, len(loader), out.loss.item(), lr)

        logits = out.logits[0].argmax(1)
        print(tokenizer.decode(logits))

    if i == 2000:
        break

costom_lora.merge(model_actor)
model_actor.save_pretrained('/content/drive/MyDrive/Simple_RLHF/model/actor')

99 7500 11.889886856079102 0.0009997532801828658
ed.
.�ATE TABLE ( (name (as (name_ARCHAR(
_ARCHAR, WHERE_ CRE is the player located?? player: Raymond playerACE( table_NAME_47 where playerER= 'RayMOND"OYD" WHEREI the the the the a the the his the a the the the the the the the the the the the the the the of the the the of the the the the the the the the the the the a the a the a the) the of a the a the the the the a the the the a the of the the  the the the much a the) the the a the the the the</s>
199 7500 3.705136299133301 0.00099778098230154
of" "_�ATE TABLE " (name ( ( (1 (name_name_table_EGER, no_season INTARCHAR, WHERE_ " many seasons have there team code had?0? the in of
 : CRE table(production_in_season) FROM table_2226817_8 WHERE production_code = 77.01"
Iance<pad><pad><pad><pad>sance<pad>onanceals<pad><pad>a<pad><pad><pad>anceance<pad>sance<pad>ance<pad><pad>aance<pad><pad><pad>os<pad>ix<pad><pad>ance<pad><pad>anceanceance<pad>anceance<pad>h<pad>ance<pad><pad><pad><pad><pad>an

In [21]:
!ls -al /content/drive/MyDrive/Simple_RLHF/model/actor

total 5139784
drwxr-xr-x 2 root root       4096 Mar  8 15:42 .
drwxr-xr-x 3 root root       4096 Mar  8 15:36 ..
-rw-r--r-- 1 root root        749 Mar  8 15:36 config.json
-rw-r--r-- 1 root root        132 Mar  8 15:36 generation_config.json
-rw-r--r-- 1 root root 4994509120 Mar  8 15:42 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  268568856 Mar  8 15:42 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root      33832 Mar  8 15:42 model.safetensors.index.json
